In [1]:
!nvidia-smi

Sat Apr 23 22:39:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!git clone --quiet https://github.com/pistocop/messaging-chat-parser.git
!pip install -q -r messaging-chat-parser/requirements.txt
!pip install aitextgen
!git clone --quiet https://github.com/pistocop/pistoBot.git

     |████████████████████████████████| 572 kB 5.3 MB/s 
     |████████████████████████████████| 4.0 MB 41.2 MB/s 
     |████████████████████████████████| 87 kB 6.4 MB/s 
     |████████████████████████████████| 582 kB 41.9 MB/s 
     |████████████████████████████████| 408 kB 46.1 MB/s 
     |████████████████████████████████| 136 kB 47.1 MB/s 
     |████████████████████████████████| 596 kB 41.6 MB/s 
     |████████████████████████████████| 1.1 MB 39.5 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |████████████████████████████████| 6.6 MB 43.0 MB/s 
     |████████████████████████████████| 895 kB 36.9 MB/s 
     |████████████████████████████████| 144 kB 50.2 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 271 kB 16.2 MB/s 
  Created wheel for aitextgen: filename=aitextgen-0.5.2-py3-none-any.whl size=575905 sha256=fa82e255994a3fa78b2da3a4275821f120eeee4069f3f9f4454c88205852a42e
  Stored in directory: /root/.cache/pi

In [28]:
%%writefile app.py

from aitextgen import aitextgen
from pprint import pprint
from datetime import datetime
from datetime import timedelta
from datetime import date
import threading
import os
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use the application default credentials
cred = credentials.Certificate("./serviceAccountKey.json")
firebase_admin.initialize_app(cred)

db = firestore.client()
collection = db.collection('users').stream() 

count = 0

class Chat:
    def __init__(self, id, autoChat, chatBotMode, date, freq, export):
        self.autoChat = autoChat
        self.chatBotMode = chatBotMode
        self.lastUpdate = date
        self.frequency = freq
        self.id = id
        self.export = export

class User:
  def __init__(self, id):
    self.convos = []
    self.id = id

def getUsers():
    users = []
    collection = db.collection('users').stream() 
    for doc in collection:
        user = User(doc.id)
        chats = db.collection(doc.id).stream()
        for chat in chats:            
            temp = chat.to_dict()
            print(temp)
            convo = Chat(chat.id, temp['autoChat'], temp['chatBotMode'], temp['lastUpdate'], temp['frequency'], temp['export'])
            user.convos.append(convo)
        users.append(user)
    return users


# def getConvos():
#     print()

def getUserNameById(id):
    doc_ref = db.collection('users').stream()
    for doc in doc_ref:
        if doc.id == id:
          temp = doc.to_dict()
          return temp['name']

def parseChats(userId, convoId):
    text_file = open("./messaging-chat-parser/data/chat_raw/whatsapp/"+userId+convoId+".txt", "w")
    doc_ref = db.collection(userId).stream()
    for doc in doc_ref:
        if doc.id == convoId:
            temp = doc.to_dict()
            text_file.write(temp['export'])
            text_file.close()
            tmp = getUserNameById(userId)
            l = tmp.split()
            user_name = ""
            for item in l:
              user_name+="\ "+item

            datetime_format = "%m/%d/%y,\ %H:%M"
            print("> start parsing...")
            assert user_name is not None, "[!] User name not available"
            cmd = "cd messaging-chat-parser && python ./src/whatsapp_parser.py --session_token "+ "\"<|endoftext|>\"" + " --user_name " + user_name + " --time_format " + datetime_format+" && " + "mv ./data/chat_parsed/wa-chats.txt ./data/chat_parsed/"+userId+convoId+".txt"
            print(cmd)
            os.system(cmd)
            print("----------------------------------")

# parseChats('9305021235', '9026663925')

def trainModel(userId, convoId):
    os.system("cp ./messaging-chat-parser/data/chat_parsed/models-trained"+userId+convoId+".txt " + "./pistoBot/data/inputs/chat_parsed/all-messages-endoftext.txt")
    os.system("cd pistoBot/colab/ && bash run_training.sh gpt2-scratch")

def loadModel():
    files = os.listdir("./pistoBot/data/models-trained")
    folder_name = files[0]
    
    model_path = os.path.join(".", "pistoBot", "data", "models-trained", folder_name)
    config_path = os.path.join(".", "pistoBot", "data", "models-trained", folder_name,"config.json")
    vocab_path = os.path.join(".", "pistoBot", "data", "models-trained", folder_name,"aitextgen-vocab.json")
    merges_path = os.path.join(".", "pistoBot", "data", "models-trained", folder_name,"aitextgen-merges.txt")

    ai = aitextgen(model_folder=model_path, 
               config=config_path,
               vocab_file=vocab_path,
               merges_file=merges_path,
               to_gpu=True)
    return ai



def addMsgToConvo(msg, senderNo, receiverNo):
    doc_ref = db.collection('chats').document()
    doc_ref.set({
        'Message Type': 'text',
        'message': msg,
        'receiverNo': receiverNo+'x',
        'senderNo': senderNo,
        'time': firestore.SERVER_TIMESTAMP
    })
    print(doc_ref.id)

# addMsgToConvo("0xCAFEBABE", "9305021235", "1234567890")

def getLastMsgFromConvo(userId, convoId):
    doc_ref = db.collection(userId).stream()
    for doc in doc_ref:
      if doc.id == (convoId+'x'):
        tmp = doc.to_dict()
        return tmp['lastMessage']

def chatBot(lastMessage, userId, convoId):
  chat = []
  start_temperature = 0.7
  max_temperature = 3.0

  for _ in range(5):
      new_line = "[others] " + lastMessage + '\n'
      chat.append(new_line)
      
      me_token = False
      temperature = start_temperature
      input_network = ' '.join(chat)
      
      while not me_token:
          ai = loadModel()
          text = ai.generate(prompt=input_network, 
                            return_as_list=True, 
                            temperature=temperature)
          text = text[0] # batch of 1

          text = text.split('\n')
          print(text)
          msg = text[2].split();
          ans = ""
          for i in range(len(msg)):
            if i==0:
              continue
            ans = ans + msg[i] + " "
          print(ans)
          return ans

print(chatBot(getLastMsgFromConvo('9305021235', '9026663925'), '9305021235', '9026663925'))
    


while True:
    if count%1000000 == 0:
        users = getUsers()
        for user in users:
            user.convos = getConvos()

    if datetime.now().strftime("%H:%M:%S") != "00:00:00":
        count = count + 1
        continue

    for user in users:
        for convo in user.convos:
            if convo.autoChat and convo.lastUpdate+timedelta(days=convo.frequency) == date.today():
                print("Model training of "+convo.id+" for "+user.id)
                t1 = threading.Thread(target=createModel, args=(convo.id,))
                t1.start()
            while convo.chatBotMode:
                print("Conversation started between "+user.id+" and "+convo.id)
                t2 = threading.Thread(target=addMsgToConvo, args=((getLastMsgFromConvo(convo.id),))
                t2.start()


    count = count + 1
    


Overwriting app.py


In [29]:
!python3 app.py

['[others] hello', '', "[others] Don't worry bro", '', '[others] whu', '', '[others] why', '', '']
Don'tworrybro
Don'tworrybro
